In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 11}
----------
[[10850  4509]
 [  410 14997]]
----------
              precision    recall  f1-score   support

           0       0.96      0.71      0.82     15359
           1       0.77      0.97      0.86     15407

    accuracy                           0.84     30766
   macro avg       0.87      0.84      0.84     30766
weighted avg       0.87      0.84      0.84     30766

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.23088181, 0.25731194, 0.32114184, 0.30917335, 0.35604692,
        0.37300205, 0.38546789, 0.42137265, 0.43334055, 0.43633163,
        0.47074127, 0.48620093, 0.52759147, 0.52808762, 0.60887134,
        0.33808351, 0.3271246 , 0.37449753, 0.44281495, 0.49517477,
        0.53606629, 0.53955555, 0.60388529, 0.63330615, 0.62083817,
        0.65524733, 0.66023421, 0.68417037, 0.63679504, 0.67070448]),
 'std_fit_time': array([0.00049794, 0.00897586, 0.01795161, 0.01196885, 0.00199366,
        0.02493286, 0.02443516, 0.01545811, 0.03739929, 0.02443397,
        0.02792549, 0.02643025, 0.03191018, 0.00249314, 0.03540576,
        0.00996304, 0.00797915, 0.01745307, 0.01496112, 0.03141463,
        0.02642941, 0.00199509, 0.00448906, 0.00698125, 0.00648284,
        0.01894891, 0.00398922, 0.00897634, 0.02642941, 0.00149596]),
 'mean_score_time': array([0.01446128, 0.01246643, 0.01346564, 0.01346338, 0.01346481,
        0.01296544, 0.01446009, 0.01496041, 0.01396239, 0.01